In [ ]:
# This file contains tests for cached methods
import sys
import gc
import warnings
warnings.filterwarnings('ignore')

import numpy as np

sys.path.insert(0, '../../../seismiqb')
sys.path.insert(0, '../../../batchflow')

from seismiqb import Field, Horizon, Geometry
from seismiqb import GlobalCache

In [ ]:
# Defaults for run this notebook directly
# Data paths (note, we use other tests data, no need in data creation)
GEOMETRY_PATH = './horizon_test_files/test_cube.sgy'
HORIZON_PATH = './horizon_test_files/test_horizon'

In [ ]:
def cached_objects_ids():
    """ Get ids of instances which use cache. """
    return [id(x) for x in GlobalCache.instances_with_cache]

In [ ]:
# Import stage check
assert len(GlobalCache.cache_references) != 0, 'Cache references weren\'t saved on import stage!'
assert len(GlobalCache.instances_with_cache) == 0, 'GlobalCache must be empty!'

# Geometry tests

In [ ]:
%%time
# Initialize geometry
geometry = Geometry.new(GEOMETRY_PATH)

In [ ]:
%%time
# Initialization check
# Using cache creates 'cache' attribute, now it doesn't exist, because we doesn't use cache
assert id(geometry) not in cached_objects_ids(), "Geometry cache must be disabled by default"
assert not hasattr(geometry, 'cache'), 'Geometry cache must not exist'

In [ ]:
%%time
# Load some cached data: by default cache is disabled
_ = geometry.load_slide(geometry.shape[0]//10)

assert id(geometry) not in cached_objects_ids(), "Geometry cache must be disabled by default"
assert not hasattr(geometry, 'cache'), 'Geometry cache must not exist'

In [ ]:
%%time
# Enable cache and load some cached data
geometry.enable_cache()

_ = geometry.load_slide(geometry.shape[0]//10)

assert id(geometry) in cached_objects_ids(), 'Geometry object must have cached data'
assert geometry.cache_size > 0, 'Geometry object must have cached data'

In [ ]:
%%time
# Load more slides: they must be saved in cache while maxsize is not reached
maxsize = GlobalCache.cache_references['Geometry.load_slide_cached'].maxsize

for i in range(maxsize + 1):
    _ = geometry.load_slide_cached(i)

assert geometry.cache_size == GlobalCache.size == maxsize, "Invalid cached objects amount"

FIFO_error = "The first added element wasn\'t replaced: the FIFO logic broken"
assert geometry.get_cache_repr()['Geometry.load_slide_cached']['arguments'][0]['index'] > 0, FIFO_error

In [ ]:
%%time
# Clear cache
GlobalCache.reset()

assert len(geometry.cache) == geometry.cache_size == GlobalCache.size == 0, 'Cache wasn\'t properly cleared'

In [ ]:
%%time
# Remove geometry object and check reference on it in the GlobalCache
geometry_id = id(geometry)

assert geometry_id in cached_objects_ids(), 'Geometry object must be saved in the GlobalCache'

del geometry
gc.collect()

assert geometry_id not in cached_objects_ids(), 'Broken reference on unexisted object in the GlobalCache'

In [ ]:
# Check that GlobalCcahe was cleared after geometry tests
assert len(GlobalCache.instances_with_cache) == 0, 'GlobalCache wasn\'t cleared'

# Horizon tests

## Data loading

In [ ]:
%%time
field = Field(geometry=GEOMETRY_PATH)
horizon = Horizon(storage=HORIZON_PATH, field=field)

In [ ]:
%%time
# Initialization check
# Using cache creates 'cache' attribute, now it doesn't exist, because we doesn't use cache
assert id(horizon) not in cached_objects_ids(), "Horizon cache must be disabled by default"
assert not hasattr(horizon, 'cache'), 'Horizon cache must not exist'

## Cache manipulations checks

In [ ]:
%%time
# Cached property check
_ = horizon.binary_matrix

horizon_cache_size = np.sum([len(x) for x in horizon.cache.values()])

assert horizon_cache_size == horizon.cache_size == 1, 'Invalid cache length for the object with one cached property'

assert len(GlobalCache.instances_with_cache) == 1, 'There is must be one reference on instance in the Global cache'

assert id(list(GlobalCache.instances_with_cache)[0]) == id(horizon), 'GlobalCache has invalid instance reference'

In [ ]:
%%time
# Check reset cache for object
horizon.reset_cache()

assert len(horizon.cache) == horizon.cache_size == GlobalCache.size == 0, 'Cache wasn\'t cleared'

In [ ]:
%%time
# Cached object check
_ = horizon.get_fourier_decomposition()

assert horizon.cache_size == GlobalCache.size > 0, 'Invalid cache length'

In [ ]:
%%time
# Check global reset cache
GlobalCache.reset()

assert len(horizon.cache) == horizon.cache_size == GlobalCache.size == 0, 'Cache wasn\'t cleared'

## Remove instances

In [ ]:
%%time
# Check cache removal after object deletion
horizon_id = id(horizon)

assert horizon_id in cached_objects_ids(), 'Object haven\'t cache'

del horizon
gc.collect()

assert horizon_id not in cached_objects_ids(), 'Broken reference in the GlobalCache'

## Multiple cached instances check

In [ ]:
%%time
horizon_1 = Horizon(storage=HORIZON_PATH, field=field)
horizon_2 = Horizon(storage=HORIZON_PATH, field=field)

_ = horizon_1.binary_matrix
_ = horizon_2.get_fourier_decomposition()

assert len(GlobalCache.instances_with_cache) == 2, 'Broken references in the GlobalCache'
assert horizon_1.cache_size + horizon_2.cache_size == GlobalCache.size > 0, 'Invalid cache length'

In [ ]:
# Check cache representation call
_ = display(horizon_1.cache_repr), display(horizon_2.cache_repr), display(GlobalCache.repr)

In [ ]:
%%time
# Clear one object cache
horizon_1.reset_cache()

assert len(horizon_1.cache) == 0, 'Cache wasn\'t cleared'
assert horizon_2.cache_size == GlobalCache.size > 0, 'Invalid cache length'

# Return cache to the previous state for next tests
_ = horizon_1.binary_matrix

In [ ]:
%%time
# Clear global cache
GlobalCache.reset()

assert horizon_1.cache_size == horizon_2.cache_size == GlobalCache.size == 0, 'Cache wasn\'t cleared'

# Return cache to the previous state for next tests
_ = horizon_1.binary_matrix
_ = horizon_2.get_fourier_decomposition()

In [ ]:
%%time
# Check cache removal after object deletion
horizon_id_to_remove = id(horizon_1)

assert horizon_id_to_remove in cached_objects_ids(), 'Object hasn\'t cache'

del horizon_1
gc.collect()

assert horizon_id_to_remove not in cached_objects_ids(), 'Broken reference in the GlobalCache'

In [ ]:
# Clear cache
del horizon_2
gc.collect()

assert len(GlobalCache.instances_with_cache) == 0, 'Cache wasn\'t properly cleared'

# Field

In [ ]:
field.load_labels(labels=HORIZON_PATH, labels_class='horizon')

assert len(field.attached_instances) == 1, 'Horizon wasn\'t loaded'

In [ ]:
# Load cached data and check field and global cache
_ = field.attached_instances[0].binary_matrix
_ = field.geometry.load_slide_cached(field.shape[0]//10)

assert field.cache_nbytes == GlobalCache.nbytes > 0, 'Invalid cache size'

In [ ]:
# Check cache reset
field.reset_cache()

assert field.cache_nbytes == GlobalCache.nbytes == 0, 'Cache wasn\'t cleared'